In [48]:
# Scrapping from reclameaqui.com.br
import pandas as pd
import time
import random
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
import os
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import setuptools._distutils


In [49]:
# nome da empresa seguindo convenção do site reclameaqui.com.br
empresa : str = 'itau'

# URL da pagina principal da empresa
url_p = 'https://www.reclameaqui.com.br/empresa/' + empresa + '/'

# URL da pagina de reclamações da empresa
def url_r(page : int, type : str = 'EVALUATED') -> str:

    # https://www.reclameaqui.com.br/empresa/temu/lista-reclamacoes/?pagina=1&status=EVALUATED
    return 'https://www.reclameaqui.com.br/empresa/' + empresa + '/lista-reclamacoes/?pagina='+ str(page) +'&status=' + type



In [ ]:
# pegando metadados da empresa (nome, categoria, reputação, sobre)

# driver
driver = uc.Chrome()

if not os.path.exists(f'metadados/metadados_{empresa}.csv'):
   # acessando a pagina principal da empresa
   driver.get(url_p)

   time.sleep(random.randint(4,6))
      
   # ramo de atividade da empresa
   categoria = driver.find_element(By.ID, 'info_segmento_hero').text

   # reputação da empresa
   reputacao = driver.find_element(By.ID, 'tag_reputacao_hero').text

   # sobre a empresa
   sobre = driver.find_element(By.CSS_SELECTOR, 'ul.sc-1915fv4-2.idgfcC').text
   sobre = sobre.split("\n")[0] 

   metadata = {
      'nome': empresa,
      'categoria': categoria,         
      'reputacao': reputacao,         
      'sobre': sobre       
   }
   
   metadata_p = pd.DataFrame(metadata, index=[0])
   metadata_p.to_csv(f'metadados/metadados_{empresa}.csv', index=False)

else:
   
   metadata = pd.read_csv(f'metadados/metadados_{empresa}.csv')
   metadata = metadata.to_dict(orient='records')[0]
   
   
print(metadata)




   nome             categoria reputacao  \
0  itau  Bancos e Financeiras       Bom   

                                               sobre  
0  O Itaú Unibanco é um dos maiores bancos privad...  


In [ ]:
# pegando hyperlinks das reclamações avaliadas e/ou respondidas

driver.get(url_r(page=1, type='EVALUATED'))

all_hyperlinks = []

Numero_de_paginas = 50 # máximo de 50 páginas

# se o csv já existir continuar, se não criar 
if not os.path.exists(f'hyperlinks/hyperlinks_{empresa}.csv'):

   for i in range(Numero_de_paginas):
      try:
         div_hyperlinks = driver.find_elements(By.CSS_SELECTOR, 'div.sc-1pe7b5t-0.eJgBOc') # div dos hyperlinks das reclamações
         
         hyperlinks = [h.find_element(By.CSS_SELECTOR, 'a').get_attribute('href') for h in div_hyperlinks] # pegando os hyperlinks
         
         print(hyperlinks)
         
         all_hyperlinks = all_hyperlinks + hyperlinks
         
         botao = driver.find_element(By.CSS_SELECTOR, "button[data-testid='next-page-navigation-button']")
         driver.execute_script("arguments[0].click();", botao)

         time.sleep(random.randint(5, 8))
         
      except:
         
         break
      
   # transformando em csv os hyperlinks
   df = pd.DataFrame(all_hyperlinks)
   df.to_csv(f'hyperlinks/hyperlinks_{empresa}.csv', index=False)
   all_hyperlinks = df.values.tolist()
      
else:
   
   # lendo o arquivo csv existente
   df = pd.read_csv(f'hyperlinks/hyperlinks_{empresa}.csv')
   all_hyperlinks = df.values.tolist()
   
print(all_hyperlinks)


   

[['https://www.reclameaqui.com.br/itau/algo-deu-errado-nao-foi-possivel-carregar-as-informacoes_7sQIuRHJxMUgzgt-/'], ['https://www.reclameaqui.com.br/itau/nao-sou-gilmara_qUVHLaF91vD1DZGV/'], ['https://www.reclameaqui.com.br/itau/apontamento-de-2009_XpybqcYKGOgEjnzm/'], ['https://www.reclameaqui.com.br/itau/senha-errada-mesmo-estando-correta_Dn_Ue3moQEXlbKDL/'], ['https://www.reclameaqui.com.br/itau/encerramento-de-conta-liberacao-de-dinheiro_6tRqmuIaYMMDPsWE/'], ['https://www.reclameaqui.com.br/itau/tarifa_TeHV9MdvYxa5FFlM/'], ['https://www.reclameaqui.com.br/itau/registro-no-scr_SzfBSBzERB7vvdq3/'], ['https://www.reclameaqui.com.br/itau/super-endividamento-credito-emergencial_UXSV-_CxQREPRFp3/'], ['https://www.reclameaqui.com.br/itau/abertura-de-conta-sem-autorizacao_yOa5fuuz-VhPJusH/'], ['https://www.reclameaqui.com.br/itau/existe-um-debito-e-nao-consigo-pagar_N_jE0nL46Ez8DbfF/'], ['https://www.reclameaqui.com.br/itau/boletos-pagos-consta-em-aberto_4f-7vZu9Tapo5fMS/'], ['https://www

In [52]:
# pegando dados das reclamações respondidas

reclamacoes = []

#for link in all_hyperlinks:
for link in all_hyperlinks[0:9]:
   
   link = link[0]
   
   try:
      driver.get(link)

      time.sleep(random.randint(5, 8))

      titulo = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-title"]').text
      lugar = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-location"]').text
      data = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-creation-date"]').text
      reclamacao_id = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-id"]').text
      
      reclamacao_id = (reclamacao_id.split('ID: ')[1])

      parsed_data = time.strptime(data, '%d/%m/%Y às %H:%M')

      reclamacao_base = driver.find_element(By.CSS_SELECTOR, '.sc-lzlu7c-17.fRVYjv').text
      interacoes = driver.find_elements(By.CSS_SELECTOR, '[data-testid="complaint-interaction"]')

      interacoes_text = [i.text for i in interacoes]

      again = driver.find_element(By.CSS_SELECTOR, '.sc-uh4o7z-0.bprQlw:nth-child(2) > div').text
      rating = driver.find_element(By.CSS_SELECTOR, '.sc-uh4o7z-0.bprQlw:nth-child(3) > div').text

      rating = int(rating.split()[0])
      again = again == 'Sim'

      reclamacao = {
         'id': reclamacao_id,
         'titulo': titulo,
         'lugar': lugar,
         'data': parsed_data,
         'reclamacao_base': reclamacao_base,
         'interacoes': interacoes_text,
         'rating': rating,
         'again': again
      }

      reclamacoes.append(reclamacao)
      

   except:
      
      continue

print(reclamacoes)

   
driver.quit()

# transformando em csv as reclamações
df = pd.DataFrame(reclamacoes)
df.to_csv(f'reclamacoes/reclamacoes_{empresa}.csv', index=False)



[{'id': '212799431', 'titulo': 'Algo deu errado. Não foi possível carregar as informações.', 'lugar': 'Alhandra - PB', 'data': time.struct_time(tm_year=2025, tm_mon=3, tm_mday=21, tm_hour=21, tm_min=43, tm_sec=0, tm_wday=4, tm_yday=80, tm_isdst=-1), 'reclamacao_base': 'Toda vez que peço um cartão aparece isso', 'interacoes': ['Resposta da empresa\n22/03/2025 às 08:27\nOlá, Gleibson, bom dia!\nEsperamos que esteja tudo bem com você.\n\nAnalisamos a sua solicitação, informamos que a disponibilização, cartão de credito, demais produtos bancários de crédito são feitas a partir de uma avaliação de seu relacionamento com o banco e de suas condições de crédito.\n\nDessa forma, tais produtos são variáveis e estão sujeitos a revisões e às normas vigentes para concessão de crédito.\n\nAnalisando os dados que temos disponíveis hoje, não foi possível enquadrá-lo em todos os nossos critérios de concessão de crédito levando em consideração a política de crédito do Banco.\n\nAs propostas são analisad